<h1 style="margin:0; line-height:1.5; padding:0;">Intro to Computational Thinking and Programming - Group 4</h1>
<p style="margin:0;line-height:1; padding:0; font-size:20px;">Team Allocation Simulator</p>
<h5 style="margin:0; margin-top:10px; line-height:1; padding:0; padding-top:10px; color:rgba(231, 60, 60, 1); text-decoration: underline; font-weight:bold;">Objective:</h5>
<ol>
    <li>School Affliation: No majority of students from the same school</li>
    <li>Gender: No majority of students of the same gender</li>
    <li>Current CGPA: To balance academic performance</li>
</ol>

<p>This block imports the necessary libraries, provides the path to the data file
and defines a variable for the teamsize to allow for further customisation</p>

<style>
ol{
    line-height:.2;
    margin-top:5px;
    padding-top:5px;
}

li{
    line-height:1.3;
}
</style>

In [2]:
import csv
import matplotlib.pyplot as plt
import json
path = "records.csv" #this is the path to the data file
teamsize = 5

<h5 style="text-decoration:underline; margin:0; padding:0; line-height:1; font-weight:bold;">Reading and Sorting Data</h5>
<p>Define a function called <b>tut_grp_json</b> that reads data from the <b>records.csv file</b> and creates a <b>tut_grp.json file</b> that contains the students details sorted based on their tutorial group in a numerical order</p>
<b style="line-height:.5; margin:0; padding:0;"><u>Pseudocode:</u></b>
<p style="line-height: 1.5; margin:0; padding:0;">Function Name: tut_grp_json</p>
<p style="line-height: 2; margin:0; padding:0;">Parameters: path, json_filename</p>
<p style="line-height: 1.3; margin:0; padding:0;">Local Variables: json_filename(name of json file), data(where student data stored), fObj(file)
, reader(data from records.csv), row(separate rows of records.csv), key(key of data dictionary)
, key_list(where list of keys stored), student_data(sorted student data stored) </p>
<ol>
    <li>Append .json to <b>json_filename</b></li>
    <li>create empty dicitonary to store data from records.csv</li>
    <li>Open and read records.csv</li>
    <li>Use DictReader, a class from csv library to read data in records.csv</li>
    <li>For each row in records.csv:</li>
    <ol>
        <li>Extract "Tutorial Group" from row to be used as key of dictionary</li>
        <ul>
            <li>If <b>key</b> not in <b>data</b>: Create new list with row as first item</li>
            <li>Else append row to existing list of the existing key</li>
        </ul>
    </ol>
    <li>Sort tutorial group numerically:</li>
    <ol>
        <li>Create an empty list(key_list) to store numerical value of tutorial groups</li>
        <ul>
            <li>Split each key in <b>data</b> by <b>"-"</b> and add the separated tutorial group number to <b>key_list</b></li>
            <li>Use the sort method to sort values in key_list in numerical order</li>
            <li>Concantate the letter G to all numerical values (e.g. G-1) in key_list</li>
        </ul>
    </ol>
    <li>Store the sorted data into a new dictionary(student_data) with tutorial groups being the key of each item of the dictionary</li>
    <li>Write student_data into tut_grp.json file</li>
    <li>Return json file back to global variable(json_file)</li>
</ol>

<style>
    ol > ol{
        list-style-type: lower-alpha;
    }
</style>

In [ ]:
def tut_grp_json(path: str,json_filename: str='student_data'): #path: takes in filepath of csv file; json_filename: specify the file name of the json file without the filetype
    json_filename=json_filename+'.json'
    data={} #overall data
    with open(path,'r',newline='') as fObj:
        reader=csv.DictReader(fObj)
        for row in reader:
            gpa=row["CGPA"]
            row["CGPA"]=float(gpa)
            key=row.pop('Tutorial Group') #remove the tutorial group from the row and store the variable
            if key not in data:
                data[key]=[row] #the value of data[key] is a list; if key not in, create new key
            else:
                data[key].append(row) #if key already exist, append to the existing list corresponding to the key
    
    ###sort students per in tut grp from highest to lowest gpa
    for key in data:
        sorted_by_gpa=sorted(data[key],key=lambda student: student['CGPA'],reverse=True) #sort all the students according to cgpa
        data[key]=sorted_by_gpa

    ###sorting out the keys to make sure they are in order
    key_list=[]
    for key,student_info in data.items():
        key_list.append(int(key.split('-')[1])) #split string at '-'.
        key_list.sort() #sort from 1-120 in numerical order because python does not sort 'G-xx' in numerical order
    def myFunc(i):
        return 'G-'+str(i)
    key_list=list(map(myFunc,key_list))
    student_data = {key: data[key] for key in key_list}

    ###writing the the student_data object into a json file
    with open(json_filename,'w') as fObj:
        #json file cannot be modifiable
        json.dump(student_data,fObj,indent=2) #write to json file so the sorted dictionary cannot be accidentally messed up
    return json_filename


<h5 style="line-height:.5; margin:0; padding:0; font-weight:bold;">Why .json?</h5>
<p>As data stored in a .json file cannot be modified. Thus, ensuring that data is not modified while being used in the program.</p>
<h5 style="text-decoration:underline; line-height:1; margin:20px 0 0 0; padding:20px 0 0 0; font-weight:bold;">Loading Sorted Data into Python Program</h5>
<p>Define a function called <b>load_json_to_dict</b> that returns sorted data from the tut_grp.json file into a global variable called values that called the function to store data in a dictionary</p>

In [4]:
def load_json_to_dict(json_file: str): #loads json file to a dictionary and returns a dictionary
    with open(json_file,'r') as jFileObj:
        values=json.load(jFileObj)
    return values

In [5]:
json_file=tut_grp_json(path)

values=load_json_to_dict(json_file)

In [ ]:
def summarise_data_json(student_data: dict, json_filename: str = 'summarised_data'): #student_data is dictionary onjects of student info sorted via tutorial group, dictionary should be loaded via json.load()  
    json_filename += '.json'
    data_set={}

    ###sum up all possible faculties and gender
    schools=set()
    genders=set()
    for tut_grp,tut_data in student_data.items(): 
        grp_set=set(map(lambda student: student['School'], tut_data))
        gender_set=set(map(lambda student: student['Gender'], tut_data))
        schools|=grp_set #keep updating the set 'schools' to check how many schools are there in total
        genders|=gender_set
    
    ###sorting the numerical data via tutorial group
    for tut_grp,tut_data in student_data.items(): #tut_data is a list of dictionary object of student info
        summary_stats={}
        ###gpa data
        gpa_list=list(map(lambda student : student["CGPA"], tut_data )) #each iterable stored in the list (dictionary of student info) will be passed thru lambda function
        summary_stats['CGPA']=gpa_list
        
        ###faculty data
        all_schools=list(map(lambda student : student['School'], tut_data))
        def count_sch(sch):
            return (sch,all_schools.count(sch))
        school_tuple=tuple(map(count_sch,schools)) #schools is a set
        school_dict=dict(school_tuple)
        summary_stats['Schools']=school_dict

        ###gender data
        all_genders=list(map(lambda student : student['Gender'], tut_data))
        def count_gender(gender):
            return (gender,all_genders.count(gender))
        gender_tuple=tuple(map(count_gender,genders)) #genders is a set
        gender_dict=dict(gender_tuple) 
        summary_stats['Genders']=gender_dict

        ###count of students
        summary_stats['Count']=len(gpa_list)

        data_set[tut_grp]=summary_stats
        #The dict() constructor builds dictionaries directly from iterable sequences of key-value pairs
        #https://docs.python.org/3/tutorial/datastructures.html#dictionaries

    
    with open(json_filename,'w') as fObj:
        json.dump(data_set,fObj,indent=2)
    
    return json_filename